In [1]:
import pandas as pd
import plotly.express as px

In [22]:
bam_counts = pd.read_csv("bam_counts.csv")
bam_counts["sample_name"] = bam_counts.apply(lambda row: f'{row["background"]}-{row["sample"]}', axis=1)
bam_counts["values"] = bam_counts.apply(lambda row: ((1 if row["aligner"]=="bwa" else -1) * row["num_reads"]), axis=1)
bam_counts

,aligner,background,sample,num_reads,sample_name,values
0,bwa,wb,Mixture01,423031,wb-Mixture01,423031
1,bwa,wb,Mixture02,2477301,wb-Mixture02,2477301
2,bwa,wb,Mixture03,625803,wb-Mixture03,625803
3,bwa,wb,Mixture04,334617,wb-Mixture04,334617
4,bwa,wb,Mixture05,876549,wb-Mixture05,876549
...,...,...,...,...,...,...
247,minimap2,pwrb,Mixture38,295957,pwrb-Mixture38,-295957
248,minimap2,pwrb,Mixture39,468959,pwrb-Mixture39,-468959
249,minimap2,pwrb,Mixture40,411376,pwrb-Mixture40,-411376
250,minimap2,pwrb,Mixture41,542897,pwrb-Mixture41,-542897


In [30]:
px.bar(bam_counts, x="sample_name", y="values", color="aligner", title="Bam read counts for each sample")

/projects/enviro_lab/software/conda/env-plot/lib/python3.11/site-packages/plotly/express/_core.py:1985: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [32]:
px.box(bam_counts, x="aligner", y="num_reads", title="Bam read counts distribution split up by tools use to align reads")

In [33]:
wide_df = pd.merge(
    bam_counts[bam_counts["aligner"]=="bwa"].rename(columns={"num_reads":"bwa"})[["sample_name","bwa"]],
    bam_counts[bam_counts["aligner"]=="minimap2"].rename(columns={"num_reads":"minimap2"})[["sample_name","minimap2"]],
)
# wide_df["difference"] = wide_df["bwa"] - wide_df["bwa"]
wide_df["percent_difference_abs"] = abs(wide_df["bwa"] - wide_df["minimap2"]) / wide_df["minimap2"] * 100
wide_df["percent_difference"] = (wide_df["bwa"] - wide_df["minimap2"]) / wide_df["minimap2"] * 100
wide_df

,sample_name,bwa,minimap2,percent_difference_abs,percent_difference
0,wb-Mixture01,423031,422837,0.045881,0.045881
1,wb-Mixture02,2477301,2476625,0.027295,0.027295
2,wb-Mixture03,625803,625041,0.121912,0.121912
3,wb-Mixture04,334617,333359,0.377371,0.377371
4,wb-Mixture05,876549,873065,0.399054,0.399054
...,...,...,...,...,...
121,pwrb-Mixture38,296174,295957,0.073321,0.073321
122,pwrb-Mixture39,469195,468959,0.050324,0.050324
123,pwrb-Mixture40,411710,411376,0.081191,0.081191
124,pwrb-Mixture41,543058,542897,0.029656,0.029656


In [39]:
px.bar(wide_df, x="sample_name", y="percent_difference", title="Percent difference in read counts (positive values correspond to bwa yielding more reads than minimap2)")

In [38]:
px.box(wide_df, y="percent_difference", title="Distribution of percent difference in read counts (positive values correspond to bwa yielding more reads than minimap2)")